<a 
href="https://colab.research.google.com/github/5af1/Pioneer-Alpha/blob/master/task3/catr.ipynb"
target="_parent">
<!--- "https://colab.research.google.com/github/saahiluppal/catr/blob/master/catr_demo.ipynb" -->
<img 
src="https://colab.research.google.com/assets/colab-badge.svg" 
alt="Open In Colab"/></a>

<!--- This is an HTML comment in Markdown -->

In [ ]:
! git clone --recursive https://github.com/5AF1/Pioneer-Alpha.git